In [1]:
from typing import Iterable
import glob
import datetime
import copy
import os, sys

import pandas as pd
import numpy as np

from clickhouse_driver import Client

root_path = os.path.realpath("../../../..")
sys.path.append(os.path.join(root_path, "clickhouse_utils/src"))

from clickhouse_utils.schema import initialize_schema


In [2]:
# define all the constants
data_dir = os.path.join(root_path, "event_model/data") + "/"

host = "172.31.50.194"
port = 9000
http_port = 8123

database_name = "fraud"
table_name = "fraud"

order_by = ("USER_ID","DATE_CREATED")
high_cardinality = []

start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 4, 21)

In [3]:
# the 'Ethinicity' column is mislableled, and actually describes nationality, ie citizen of what state the client is
data = pd.read_csv(data_dir + f"{table_name}/fraud-2021-03.csv").rename(columns={'ETHNICITY':'NATIONALITY'})
data['DATE_CREATED'] = pd.to_datetime(data['DATE_CREATED'])

In [4]:
display(data.head())
display(data.dtypes)
display(len(data))

,USER_ID,NATIONALITY,DATE_CREATED,STATE
0,23569214,\N,2021-03-16 15:39:42.393340,NEW
1,23569214,aus,2021-03-24 15:21:49.007797,NEW
2,23569214,aus,2021-03-28 16:34:32.318904,NEW
3,23523345,\N,2021-03-01 07:15:25.019546,NEW
4,23523345,\N,2021-03-02 07:08:10.525745,NEW


USER_ID                  int64
NATIONALITY             object
DATE_CREATED    datetime64[ns]
STATE                   object
dtype: object

23677329

In [5]:
# create the schema, inferring it from dataframe
client = Client(host=host, port=port)

# read in a sample dataframe to create the schema
initialize_schema(
    client,
    database_name=database_name,
    table_name=table_name,
    dtypes=data.dtypes,
    order_by = order_by,
    high_cardinality= high_cardinality,
    flush_table=True,
)



                CREATE TABLE IF NOT EXISTS fraud.fraud
                (
                    `USER_ID` Int64,
    `NATIONALITY` LowCardinality(String),
    `DATE_CREATED` DateTime64(3),
    `STATE` LowCardinality(String)
                )
                ENGINE = MergeTree()
                ORDER BY (`USER_ID`, `DATE_CREATED`)
            


In [13]:
%%time
# insert the data into the table we just created
# NOTE: THIS GOES MUCH MUCH FASTER FROM EG CSV FILES USING THE CLICKHOUSE COMMAND_LINE CLIENT
# THIS IS FOR DEMONSTRATION ONLY
from sqlalchemy import create_engine
import datetime

ch_url = f"clickhouse+native://{host}/{database_name}"
ch_engine = create_engine(ch_url, echo=False)

print(datetime.datetime.now())
for i in range(6):
    print(i)
    df = data.to_sql('fraud', ch_engine, if_exists='append', index=False)
    print(datetime.datetime.now())


2021-04-23 11:18:21.769130
0
2021-04-23 11:23:52.222982
1
2021-04-23 11:29:28.760455
2
2021-04-23 11:35:01.619369
3
2021-04-23 11:40:35.517196
4
2021-04-23 11:46:13.329757
5
2021-04-23 11:51:45.334687
CPU times: user 32min 23s, sys: 1min, total: 33min 24s
Wall time: 33min 23s


In [22]:
client.execute(f"select count(*) from  {database_name}.{table_name}")

[(189418632,)]

In [23]:
%%time
# check that the table is there
group_col = "USER_ID"
client.execute(f"select {group_col}, count(*) as cnt from {database_name}.{table_name} group by {group_col}  order by cnt desc limit 100")


CPU times: user 4.81 ms, sys: 155 µs, total: 4.97 ms
Wall time: 951 ms


[(21059557, 1869592),
 (6240973, 1614712),
 (22415010, 1099024),
 (5568329, 351640),
 (24824103, 318992),
 (6152280, 218048),
 (2450920, 210104),
 (23134279, 184192),
 (6266878, 147944),
 (11071443, 134928),
 (18192710, 123384),
 (23126505, 106368),
 (61227, 99352),
 (15547762, 85936),
 (6844989, 77312),
 (25245086, 67024),
 (23892293, 64912),
 (4432876, 63848),
 (11971884, 63000),
 (20793030, 52864),
 (17773356, 50448),
 (10892492, 50016),
 (9728530, 48640),
 (19736173, 48088),
 (13097884, 47808),
 (4694060, 47176),
 (5487273, 46504),
 (15504265, 42912),
 (25177266, 42512),
 (4167573, 41920),
 (5838, 41512),
 (9533832, 40536),
 (21866717, 40168),
 (17566639, 36192),
 (9591155, 33552),
 (18686410, 31352),
 (14649629, 31128),
 (8970975, 30752),
 (4483753, 30720),
 (21171350, 28168),
 (4644110, 27808),
 (22886891, 26320),
 (15073285, 25672),
 (14053381, 25608),
 (14660671, 24816),
 (20486714, 24256),
 (24142812, 24120),
 (20360098, 23664),
 (7479446, 23656),
 (7069414, 22616),
 (12034306

In [24]:
%%time
# check that the table is there
group_col = "NATIONALITY, STATE"
client.execute(f"select {group_col}, count(*) as cnt from {database_name}.{table_name} group by {group_col}  order by cnt desc")


CPU times: user 7.79 ms, sys: 251 µs, total: 8.05 ms
Wall time: 2.65 s


[('\\N', 'NEW', 104385384),
 ('\\N', 'APPROVED', 14527736),
 ('bra', 'NEW', 9360176),
 ('aus', 'NEW', 8416288),
 ('deu', 'NEW', 6299800),
 ('ind', 'NEW', 6043936),
 ('esp', 'NEW', 4615992),
 ('fra', 'NEW', 4345160),
 ('ita', 'NEW', 2756568),
 ('tur', 'NEW', 1685064),
 ('sgp', 'NEW', 1522712),
 ('jpn', 'NEW', 1439232),
 ('aus', 'APPROVED', 1218912),
 ('hun', 'NEW', 1171952),
 ('pol', 'NEW', 1151560),
 ('bra', 'APPROVED', 1026512),
 ('dnk', 'NEW', 897792),
 ('pak', 'NEW', 844720),
 ('rou', 'NEW', 838392),
 ('ind', 'APPROVED', 779464),
 ('deu', 'APPROVED', 756864),
 ('bgr', 'NEW', 725352),
 ('nld', 'NEW', 720856),
 ('chn', 'NEW', 706488),
 ('ukr', 'NEW', 690032),
 ('\\N', 'FRAUD', 672760),
 ('idn', 'NEW', 638632),
 ('esp', 'APPROVED', 623712),
 ('fra', 'APPROVED', 611128),
 ('cze', 'NEW', 608112),
 ('nga', 'NEW', 516792),
 ('mys', 'NEW', 510152),
 ('irl', 'NEW', 475568),
 ('tha', 'NEW', 406024),
 ('hkg', 'NEW', 343624),
 ('ita', 'APPROVED', 341760),
 ('ukr', 'APPROVED', 301280),
 ('phl', 

In [ ]:
# sys.path.append(os.path.join(root_path, "ml_utils/src"))
# sys.path.append(
#     os.path.join(root_path, "analysts-airflow/workspace-analysts/plugins/")
# )
# from snowflake_utils.local_snowflake_wrapper import LocalSnowflakeWrapper
# from ml_utils.data.query import caching_query

In [ ]:
# def data_for_date(db_io, date: datetime.date) -> pd.DataFrame:
#     query = f"""
#         SELECT td.user_id, td.input_data:recipientNameEthnicity::STRING, td.date_created, us.state
#     FROM public_smurfs_clear.transfer_data td
#     LEFT JOIN fraud_state.user_state us ON us.user_id = td.user_id
#     WHERE td.date_created::date = '{date}';
#     """
#     return db_io.fetch(query)

In [ ]:
# # get the data, store it in daily zipped segments - only need to do it the first time round

# db_io = LocalSnowflakeWrapper("egor.kraev@transferwise.com")

# this_date = copy.copy(end_date)

# while this_date >= start_date:
#     this_df = caching_query(
#         data_dir + f"{table_name}/{table_name}_{this_date}.zip",
#         lambda: data_for_date(db_io, this_date),
#     )
#     if this_date == start_date:
#         display(this_df.head().T)
#     this_date -= datetime.timedelta(days=1)
# from clickhouse_utils.inserter import files_to_df_iterable, clickhouse_insert_dataframes
